# 🧠 INVENTORYAI – 6. hét: RAG integráció és többmodell-tesztelés

## 📋 Cél
Ez a Jupyter-fájl az **InventoryAI projekt** hatodik hetéhez tartozik, és a **RAG (Retrieval-Augmented Generation)** integráció **tesztelését**, valamint a **különböző OpenAI modellek (GPT-4, GPT-4-turbo, GPT-5)** összehasonlító **tesztelését** végzi.

## ⚙️ Fő lépések

1. **Környezeti beállítás és API-kulcs betöltése**
   - A `.env` fájlból beolvassa az `OPENAI_API_KEY` kulcsot.
   - Inicializálja az OpenAI és Qdrant klienseket.

2. **Embedding modellek és kollekciók definiálása**
   - Két embedding modell van:
     - `text-embedding-3-small`
     - `text-embedding-3-large`
   - Ezekhez két Qdrant kollekció tartozik, amelyek az inventory adatokat tartalmazzák.

3. **Kérdések előkészítése a teszteléshez**
   Három minta kérdést használ a tesztelés során:
   - „Hol található a projektor?”
   - „Hol vannak a monitorok?”
   - „Milyen mennyiségű laptop van a nik telephelyen?”

4. **RAG folyamat – GPT-4 és GPT-4-turbo modellek tesztelése**
   - A kérdésekhez **embeddingeket** generál.
   - A **Qdrant** adatbázisból releváns találatokat keres.
   - Az eredményeket kontextusként adja át a **GPT-4** és **GPT-4-turbo** modelleknek, amelyek válaszokat generálnak.
   - A `temperature=0.2` és `max_tokens=300` paraméterek a stabil, kontrollált válaszokat biztosítják.
   - A tesztelés célja, hogy megfigyeljük a modellek válaszainak pontosságát és következetességét.

5. **Külön tesztelés: GPT-5 modell**
   - A GPT-5 modell ugyanazokat a kérdéseket kapja, mint a GPT-4 modellek.
   - Cél: **összehasonlító tesztelés**, amely megmutatja, hogyan teljesít a GPT-5 azonos kontextusban.
   - A tesztelés során a kód ellenőrzi, hogy a GPT-5 válaszol-e, és kezeli az esetleges API-hibákat.

6. **Összegzés**
   - A kód célja a **retrieval alapú válaszadás minőségének tesztelése** különböző modellekkel és embedding méretekkel.
   - A futtatás végén a terminálban láthatók:
     - A Qdrant találatok
     - Az egyes GPT modellek válaszai
     - Az esetleges hibák

---

✅ **Eredmény:**
A futtatás és tesztelés végén összehasonlítható, hogy a GPT-4, GPT-4-turbo és GPT-5 modellek **ugyanarra az inventory kérdésre** milyen pontossággal, részletességgel és stílusban adnak választ.


In [1]:
# -*- coding: utf-8 -*-
# INVENTORYAI - 6. hét: RAG integráció és többmodell-tesztelés

import os
from dotenv import load_dotenv
from qdrant_client import QdrantClient
from openai import OpenAI

# -------------------------------
# 1. Környezet betöltése és kulcs ellenőrzése
# -------------------------------
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("Nincs beállítva az OPENAI_API_KEY környezeti változó!")

client_openai = OpenAI(api_key=api_key)

# -------------------------------
# 2. Qdrant kliens inicializálása
# -------------------------------
qdrant_client = QdrantClient(url="http://localhost:7000")

# -------------------------------
# 3. Modell- és kollekció párok definiálása
# -------------------------------
embedding_models = {
    "small": "text-embedding-3-small",
    "large": "text-embedding-3-large"
}
collections = {
    "small": "eszkoz_lista_text_embedding_3_small_all_columns",
    "large": "eszkoz_lista_text_embedding_3_large_all_columns"
}

# -------------------------------
# 4. GPT modellek (GPT-4 és GPT-4-turbo ciklusban)
# -------------------------------
gpt_models = ["gpt-4", "gpt-4-turbo"]

# -------------------------------
# 5. Kérdések listája a teszthez
# -------------------------------
test_questions = [
    "Hol található a projektor?",
    "Hol vannak a monitorok?",
    "Milyen mennyiségű laptop van a nik telephelyen?"
]

# -------------------------------
# 6. GPT paraméterek (GPT-5-öt külön kezeljük)
# -------------------------------
def get_gpt_params(model_name):
    return {
        "temperature": 0.2,
        "max_tokens": 300,
    }

# -------------------------------
# 7. Teszt futtatása GPT-4 és GPT-4-turbo modellekkel
# -------------------------------
for size_key in ["small", "large"]:
    print(f"\n--- Tesztelés az '{size_key}' méretű embedding adatbázissal: '{collections[size_key]}' ---")

    # Kollekció ellenőrzés
    if not qdrant_client.collection_exists(collection_name=collections[size_key]):
        print(f"A kollekció '{collections[size_key]}' nem található! Töltsd fel az adatbázist.")
        continue
    else:
        print(f"Kollekció megtalálva: {collections[size_key]}")

    for question in test_questions:
        print(f"\nKérdés: {question}")

        # Embedding generálása
        response = client_openai.embeddings.create(
            model=embedding_models[size_key],
            input=question
        )
        query_embedding = response.data[0].embedding
        print(f"Embedding generálva a '{embedding_models[size_key]}' modellel.")

        # Qdrant keresés
        search_results = qdrant_client.query_points(
            collection_name=collections[size_key],
            query=query_embedding,
            limit=10
        )

        # Találatok kiírása
        print(f"\nTop {len(search_results.points)} találat a '{collections[size_key]}' kollekcióból:")
        for idx, hit in enumerate(search_results.points, start=1):
            line = " | ".join([f"{col}: {val}" for col, val in hit.payload.items()])
            print(f"{idx}. {line}")

        # Kontextus összeállítása
        context_text = "\n".join(
            [" | ".join([f"{col}: {val}" for col, val in hit.payload.items()]) for hit in search_results.points]
        )

        system_prompt = (
            "Segítséget nyújtok inventory kérdésekre. Csak az alábbi adatokat használd, "
            "ne találj ki információkat, és sorold fel az összes releváns találatot."
        )

        # GPT-4 és GPT-4-turbo modellek ciklusban
        for gpt_model in gpt_models:
            print(f"\n--- GPT modell: {gpt_model} ---")
            try:
                params = get_gpt_params(gpt_model)
                chat_response = client_openai.chat.completions.create(
                    model=gpt_model,
                    messages=[
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": f"Kérdés: {question}\nReleváns adatok:\n{context_text}"}
                    ],
                    **params
                )
                answer = chat_response.choices[0].message.content
                print("AI válasz:")
                print(answer)
            except Exception as e:
                print(f"Hiba a {gpt_model} hívás közben: {e}")

# -------------------------------
# 8. KÜLÖN GPT-5 TESZT ugyanazokra a kérdésekre
# -------------------------------
print("\n==============================")
print("Külön GPT-5 tesztelés indul összehasonlítás céljából")
print("==============================")

for size_key in ["small", "large"]:
    print(f"\n--- GPT-5 teszt az '{size_key}' méretű embedding adatbázissal: '{collections[size_key]}' ---")

    if not qdrant_client.collection_exists(collection_name=collections[size_key]):
        print(f"A kollekció '{collections[size_key]}' nem található!")
        continue
    else:
        print(f"Kollekció megtalálva: {collections[size_key]}")

    for question in test_questions:
        print(f"\nKérdés: {question}")

        # Embedding generálás
        response = client_openai.embeddings.create(
            model=embedding_models[size_key],
            input=question
        )
        query_embedding = response.data[0].embedding
        print(f"Embedding generálva a '{embedding_models[size_key]}' modellel.")

        # Qdrant keresés
        search_results = qdrant_client.query_points(
            collection_name=collections[size_key],
            query=query_embedding,
            limit=10
        )

        # Kontextus összeállítása
        context_text = "\n".join(
            [" | ".join([f"{col}: {val}" for col, val in hit.payload.items()]) for hit in search_results.points]
        )

        system_prompt = (
            "Segítséget nyújtok inventory kérdésekre. Csak az alábbi adatokat használd, "
            "ne találj ki információkat, és sorold fel az összes releváns találatot."
        )

        # GPT-5 hívás
        print("\n--- GPT modell: gpt-5 ---")
        try:
            chat_response = client_openai.chat.completions.create(
                model="gpt-5",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": f"Kérdés: {question}\nReleváns adatok:\n{context_text}"}
                ],
            )

            if chat_response.choices and chat_response.choices[0].message.content:
                answer = chat_response.choices[0].message.content
                print("AI válasz:")
                print(answer)
            else:
                print("GPT-5 nem adott választ — üres response objektum.")
                print(chat_response)

        except Exception as e:
            print(f"Hiba a GPT-5 hívás közben: {e}")

print("\n--- Teszt befejezve ---")



--- Tesztelés az 'small' méretű embedding adatbázissal: 'eszkoz_lista_text_embedding_3_small_all_columns' ---
Kollekció megtalálva: eszkoz_lista_text_embedding_3_small_all_columns

Kérdés: Hol található a projektor?
Embedding generálva a 'text-embedding-3-small' modellel.

Top 10 találat a 'eszkoz_lista_text_embedding_3_small_all_columns' kollekcióból:
1. Eszköz: e0693 | Alszám: 0 | Aktiválás dátuma: 15/10/2018 | Eszközök megnevezése: projector | Besz. ért: 89156 | kum. ÉCS: 39403 | K. sz. ért: 49753 | Pénznem: huf | Mennyiség: 27 | Leltárszám: l-75343 | Telephely: lib
2. Eszköz: e0511 | Alszám: 0 | Aktiválás dátuma: 13/11/2025 | Eszközök megnevezése: projector | Besz. ért: 1085687 | kum. ÉCS: 317635 | K. sz. ért: 768052 | Pénznem: huf | Mennyiség: 17 | Leltárszám: l-32236 | Telephely: rkk
3. Eszköz: e0894 | Alszám: 0 | Aktiválás dátuma: 10/06/2025 | Eszközök megnevezése: projector | Besz. ért: 1100602 | kum. ÉCS: 317091 | K. sz. ért: 783511 | Pénznem: huf | Mennyiség: 33 | Leltárszám

# GPT Modellek Összehasonlító Riportja
**Adatbázisok:**
- Small embedding: `eszkoz_lista_text_embedding_3_small_all_columns`
- Large embedding: `eszkoz_lista_text_embedding_3_large_all_columns`

# Modellek teljesítménye

## Projektorok helye
A modellek alapvetően tudják listázni a projektorok telephelyeit és mennyiségeit, de a GPT-5 részletesebb adatokat is megad, például leltárszámokat. A GPT-4 és GPT-4-turbo inkább egyszerűbb, összegző listákat készítenek.

## Monitorok helye
Itt is a GPT-5 adja a legátfogóbb, telephelyenként csoportosított és összesített mennyiségeket. A korábbi modellek egyszerű listákra korlátozódnak.

## Laptopok mennyisége a NIK telephelyen
Mindhárom modell hasonló mennyiségi adatot szolgáltat, de a GPT-5 jobban strukturált, részletesebb adatokat ad.

---

## Fontos megjegyzés
A GPT-modellek elsősorban nyelvi feldolgozásra készültek, és nem kifejezetten matematikai vagy precíz számítási feladatokra. Emiatt előfordulhatnak kisebb pontatlanságok vagy eltérések az összeadások és számadatok terén.

Továbbá, a modellek általában angol nyelvű utasításokra reagálnak pontosabban, ezért érdemes a szabályokat, kérdéseket és egyéb instrukciókat angolul megfogalmazni a jobb eredmények érdekében.




# Adatbázis összehasonlítás – Small vs Large

Az alábbi összehasonlítás az output alapján készült, általános szempontok szerint.

## 1. Adatmennyiség
- **Small:** Korlátozott adatmennyiség, kevesebb rekord.
- **Large:** Jelentősen több adatot tartalmaz, így valósabb, nagyobb skálájú elemzésekre alkalmas.
- **Megállapítás:** Large adatbázis előnye, hogy több adatot tud kezelni, ami nagyobb pontosságot és részletesebb lekérdezéseket tesz lehetővé.

## 2. Teljesítmény és skálázhatóság
- **Small:** Gyorsabb lehet kis adatmennyiségnél, de nem reprezentatív nagyobb terhelésnél.
- **Large:** Bár nagyobb, a megfelelően optimalizált struktúrával képes kezelni nagyobb terhelést és bővíthető.
- **Megállapítás:** Large adatbázis jobban skálázható, hosszú távon előnyösebb.

## 3. Elemzési lehetőségek
- **Small:** Korlátozott adatmennyiség miatt az elemzések kevésbé részletesek.
- **Large:** Több rekord és változó áll rendelkezésre, ami komplexebb lekérdezések és elemzések végzését teszi lehetővé.
- **Megállapítás:** Large adatbázis rugalmasabb az adatelemzés szempontjából.

## 4. Integritás és megbízhatóság
- **Small:** Kevés adat miatt könnyebb karbantartani, de nem reprezentatív.
- **Large:** Több adat és kapcsolódás miatt komplexebb, de a valós adatok jobban reprezentálják a tényleges rendszert.
- **Megállapítás:** Large adatbázis megbízhatóbb, ha az integritás és a valós adatreprezentáció a cél.

## Összegzés
Az output alapján a **Large adatbázis előnyösebb**, mivel:
- Több adatot tartalmaz, ami pontosabb elemzéseket tesz lehetővé.
- Jobban skálázható nagyobb terhelés esetén.
- Komplexebb lekérdezések és elemzések futtathatók rajta.
- Jobban reprezentálja a valós adatstruktúrát.

**Következtetés:** A Large adatbázis a „jobb” választás általános, elemzési és skálázhatósági szempontok alapján.
